This notebook is an introduction in how to use the Tensorflow framework.

We will cover the components needed for a simple feedfoward neural network.

Key concepts & Tensorflow paradigms covered:
- Placeholders
- Variable
- Initializers
- Sessions

In [1]:
import numpy as np
import tensorflow as tf

## Tensorflow Graph Construction

In [2]:
#  we use a set of tuples to define the network structure
input_shape =(4,)  #  the length of the input numpy array
input_nodes= (6,)  #  the number of nodes in the input layer
hidden_nodes = (8,)  
output_shape = (4,)  #  num. nodes in output layer and the length of the output numpy array

The first Tensorflow paradigm is the **Placeholder**.

Tensorflow uses placeholders to feed data into the network.

The placeholder is fed using numpy arrays.

The first dimension is the number of samples in the batch - we use None to be able to input any batch size we want.

The second dimension is the shape of one sample - i.e. the length of the input numpy array.

In [3]:
network_input = tf.placeholder(tf.float32, shape=(None, *input_shape), name='network_input')

The next two Tensorflow paradigms are the **Variable** and **Initializer**.  

Variable objects are used to hold tensors of variable that tensorflow can change.  Both weights and biases are tf.Variables.

We also need to tell Tensorflow what initial values we want our varibles to be.  

To do the initialization we pass in the tf.random_normal initializer.  The shape of the Variable tensor is specified in the intializer.

The * operation is used to unpack the tuples.

In [4]:
input_weights = tf.Variable(tf.random_normal(shape=(*input_shape, *input_nodes), name='input_weights'))

We use the same pattern for the biases.

In [5]:
input_bias = tf.Variable(tf.zeros(shape=(*input_nodes,)), name='input_bias')

Now we can form the input layer using matrix multiplication between the input & weights, then add the biases.

In [6]:
pre_activation = tf.add(tf.matmul(network_input, input_weights), input_bias)

Finally we can form the layer by squeezing the output through a rectified linear unit.

In [7]:
input_layer = tf.nn.relu(pre_activation, name='input_layer_output')

We create a hidden layer using the same logic.

In [8]:
h_w = tf.Variable(tf.random_normal((*input_nodes, *hidden_nodes)), name='hidden_weights')
h_b = tf.Variable(tf.zeros((*hidden_nodes, ), name='hidden_bias'))
hidden_layer = tf.nn.relu(tf.add(tf.matmul(input_layer, h_w), h_b))

The output layer has no activation function (aka a linear activation function).  This allows the network to output negative values.

In [9]:
o_w = tf.Variable(tf.random_normal((*hidden_nodes, *output_shape)), name='output_weights')
o_b = tf.Variable(tf.zeros((*output_shape, ), name='output_bias'))
output = tf.add(tf.matmul(hidden_layer, o_w), o_b)

All of the code above allows us to make predictions with our network - i.e. we can do forward passes across the network.

Below we will setup the code for training.

We first need another placeholder.  This will serve as the target value for the network (aka y_train) - what our network should be outputting.

In [10]:
target = tf.placeholder(tf.float32, shape=(None, *output_shape), name='target')

In [11]:
loss = tf.losses.mean_squared_error(target, output, scope='loss_function')

We need an optimizer to do the heavy lifting of training the network.  

Here we use the Adam optimizer.  Note that the input here of learning rate is one of the most important in training any neural network.

In [12]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)

And finally a Tensorflow operation to actually do the training.

In [13]:
train_op = optimizer.minimize(loss)

Now all of the machinery to do forward passes and to back propagate error is in place.  

Add in a few Tensorflow summary operations to track what is going on in our network.

In [14]:
tf.summary.tensor_summary('input', network_input)
tf.summary.histogram('input_weights', input_weights)
tf.summary.histogram('output_bias', o_b)
tf.summary.scalar('loss', loss)
merged = tf.summary.merge_all()

## Running a Tensorflow Session

The first thing our model needs is data.  The function below generates training data for a simple function.

In [17]:
def generate_data(num_samples):
    """
    Generates training data for our network.

    args
        num_samples (int)

    returns
        net_in (np.array)  fed into the network (aka features or x)
        net_out (np.array)  aka target or y - the value we are trying to approx.
    """
    net_in = np.random.rand(num_samples, *input_shape)
    net_out = net_in + 10

    return net_in, net_out

#  run the function to get data to train with
inputs, targets = generate_data(100)

We now introduce another key Tensorflow paradigm - the Session.

You can think of a Session as one instance of the model.  

In [18]:
with tf.Session() as sess:
    
    #  first a necessary bit of Tensorflow boiler plate - initializing variables
    #  we do this operation using the Session
    sess.run(tf.global_variables_initializer())
    
    #  we create a FileWriter object to write out summarys to a file for Tensorboard to read
    writer = tf.summary.FileWriter('./logs', graph=sess.graph)
    
    #  now the training loop
    #  we are not splitting our training data into batches
    
    for train_step in range(100):
        #  now we run the tensorflow graph
        #  the graph is run by calling the .run method on the session
        #  the run method takes two inputs:
        #   fetches = the tf operations to run
        #   feed_dict = values for the placeholders

        #  here we fetch two operations
        #   train_op - the operation to train the network
        #   summary - the summary operations for the graph
        fetches = [loss, train_op, merged]

        #  the feed_dict is a dictionary with
        #   keys = the placeholders
        #   values = the numpy arrays
        #   note that we feed in multiple samples
        feed_dict = {network_input: inputs,
                     target: targets}
        #  finally we run the session using the fetches and feed_dict
        loss_value, _, summary = sess.run(fetches, feed_dict)
        #  the operation to add the summary to the tensorboard output file
        writer.add_summary(summary, train_step)
        print('step {} loss {}'.format(train_step, loss_value))        
        
    #  now training is done
    #  generate a test set 
    test_in, test_out = generate_data(5)

    #  here we get predictions from our network - we don't train
    pred = sess.run(output, {network_input: test_in})
    print(pred)
    print(test_out)

step 0 loss 164.50222778320312
step 1 loss 163.87179565429688
step 2 loss 163.2452392578125
step 3 loss 162.623046875
step 4 loss 162.00572204589844
step 5 loss 161.3926544189453
step 6 loss 160.78384399414062
step 7 loss 160.17982482910156
step 8 loss 159.5801239013672
step 9 loss 158.9837646484375
step 10 loss 158.39132690429688
step 11 loss 157.8040313720703
step 12 loss 157.2203826904297
step 13 loss 156.64027404785156
step 14 loss 156.06370544433594
step 15 loss 155.49107360839844
step 16 loss 154.92286682128906
step 17 loss 154.35939025878906
step 18 loss 153.79937744140625
step 19 loss 153.2428741455078
step 20 loss 152.69163513183594
step 21 loss 152.1458740234375
step 22 loss 151.60366821289062
step 23 loss 151.0650177001953
step 24 loss 150.53126525878906
step 25 loss 150.00213623046875
step 26 loss 149.47796630859375
step 27 loss 148.95750427246094
step 28 loss 148.4425048828125
step 29 loss 147.93228149414062
step 30 loss 147.42604064941406
step 31 loss 146.92401123046875
s

step 266 loss 64.47679901123047
step 267 loss 64.0664291381836
step 268 loss 63.65703201293945
step 269 loss 63.25095748901367
step 270 loss 62.84831237792969
step 271 loss 62.447635650634766
step 272 loss 62.04997634887695
step 273 loss 61.655460357666016
step 274 loss 61.26431655883789
step 275 loss 60.87666320800781
step 276 loss 60.49245071411133
step 277 loss 60.11139678955078
step 278 loss 59.7333984375
step 279 loss 59.35737228393555
step 280 loss 58.98282241821289
step 281 loss 58.61106872558594
step 282 loss 58.242156982421875
step 283 loss 57.87646484375
step 284 loss 57.513771057128906
step 285 loss 57.15400695800781
step 286 loss 56.7971076965332
step 287 loss 56.44303894042969
step 288 loss 56.091739654541016
step 289 loss 55.7431526184082
step 290 loss 55.3975830078125
step 291 loss 55.054649353027344
step 292 loss 54.714256286621094
step 293 loss 54.376747131347656
step 294 loss 54.0417366027832
step 295 loss 53.709102630615234
step 296 loss 53.37724304199219
step 297 lo

step 528 loss 11.886310577392578
step 529 loss 11.81689739227295
step 530 loss 11.748039245605469
step 531 loss 11.679758071899414
step 532 loss 11.6119966506958
step 533 loss 11.544760704040527
step 534 loss 11.478033065795898
step 535 loss 11.411808967590332
step 536 loss 11.346076965332031
step 537 loss 11.280866622924805
step 538 loss 11.216233253479004
step 539 loss 11.152105331420898
step 540 loss 11.088461875915527
step 541 loss 11.025334358215332
step 542 loss 10.962760925292969
step 543 loss 10.900653839111328
step 544 loss 10.839005470275879
step 545 loss 10.777811050415039
step 546 loss 10.717065811157227
step 547 loss 10.656757354736328
step 548 loss 10.596890449523926
step 549 loss 10.537450790405273
step 550 loss 10.47830581665039
step 551 loss 10.419198989868164
step 552 loss 10.360466003417969
step 553 loss 10.302109718322754
step 554 loss 10.244128227233887
step 555 loss 10.186635971069336
step 556 loss 10.129648208618164
step 557 loss 10.073090553283691
step 558 loss 

step 786 loss 3.9804675579071045
step 787 loss 3.97078800201416
step 788 loss 3.9611759185791016
step 789 loss 3.951631546020508
step 790 loss 3.9421563148498535
step 791 loss 3.9327476024627686
step 792 loss 3.923405170440674
step 793 loss 3.914127826690674
step 794 loss 3.904916286468506
step 795 loss 3.895770311355591
step 796 loss 3.8866894245147705
step 797 loss 3.877671480178833
step 798 loss 3.8687167167663574
step 799 loss 3.859823703765869
step 800 loss 3.8509936332702637
step 801 loss 3.8422253131866455
step 802 loss 3.8335182666778564
step 803 loss 3.82487154006958
step 804 loss 3.8162853717803955
step 805 loss 3.8077588081359863
step 806 loss 3.799290180206299
step 807 loss 3.7908811569213867
step 808 loss 3.7825310230255127
step 809 loss 3.774237632751465
step 810 loss 3.766000986099243
step 811 loss 3.7578213214874268
step 812 loss 3.7496979236602783
step 813 loss 3.7416293621063232
step 814 loss 3.7336173057556152
step 815 loss 3.725659132003784
step 816 loss 3.717755794

step 1046 loss 2.7005081176757812
step 1047 loss 2.6977288722991943
step 1048 loss 2.694956064224243
step 1049 loss 2.6921911239624023
step 1050 loss 2.6894335746765137
step 1051 loss 2.686685085296631
step 1052 loss 2.6839439868927
step 1053 loss 2.6812100410461426
step 1054 loss 2.6784842014312744
step 1055 loss 2.6757659912109375
step 1056 loss 2.6730542182922363
step 1057 loss 2.6703503131866455
step 1058 loss 2.667654037475586
step 1059 loss 2.6649651527404785
step 1060 loss 2.6622838973999023
step 1061 loss 2.6596102714538574
step 1062 loss 2.6569430828094482
step 1063 loss 2.654282331466675
step 1064 loss 2.651630163192749
step 1065 loss 2.648983955383301
step 1066 loss 2.6463451385498047
step 1067 loss 2.6437127590179443
step 1068 loss 2.641087532043457
step 1069 loss 2.6384692192077637
step 1070 loss 2.6358578205108643
step 1071 loss 2.633251905441284
step 1072 loss 2.6306540966033936
step 1073 loss 2.6280622482299805
step 1074 loss 2.625476598739624
step 1075 loss 2.622897624

...

step 8874 loss 0.022564608603715897
step 8875 loss 0.022563420236110687
step 8876 loss 0.02256222814321518
step 8877 loss 0.022561021149158478
step 8878 loss 0.022559836506843567
step 8879 loss 0.022558657452464104
step 8880 loss 0.022557459771633148
step 8881 loss 0.02255627140402794
step 8882 loss 0.022555088624358177
step 8883 loss 0.022553903982043266
step 8884 loss 0.022552723065018654
step 8885 loss 0.022551536560058594
step 8886 loss 0.022550344467163086
step 8887 loss 0.022549167275428772
step 8888 loss 0.022547975182533264
step 8889 loss 0.022546805441379547
step 8890 loss 0.02254561334848404
step 8891 loss 0.022544436156749725
step 8892 loss 0.02254326082766056
step 8893 loss 0.022542081773281097
step 8894 loss 0.022540900856256485
step 8895 loss 0.02253972925245762
step 8896 loss 0.02253853715956211
step 8897 loss 0.022537359967827797
step 8898 loss 0.022536201402544975
step 8899 loss 0.022535018622875214
step 8900 loss 0.022533850744366646
step 8901 loss 0.02253266610205173

step 9129 loss 0.022281670942902565
step 9130 loss 0.02228059247136116
step 9131 loss 0.02227950654923916
step 9132 loss 0.022278420627117157
step 9133 loss 0.022277340292930603
step 9134 loss 0.0222762580960989
step 9135 loss 0.02227516658604145
step 9136 loss 0.022274075075984
step 9137 loss 0.022273002192378044
step 9138 loss 0.022271905094385147
step 9139 loss 0.022270822897553444
step 9140 loss 0.02226974256336689
step 9141 loss 0.022268662229180336
step 9142 loss 0.022267580032348633
step 9143 loss 0.022266492247581482
step 9144 loss 0.022265411913394928
step 9145 loss 0.022264327853918076
step 9146 loss 0.022263234481215477
step 9147 loss 0.022262172773480415
step 9148 loss 0.022261084988713264
step 9149 loss 0.02226000279188156
step 9150 loss 0.02225891314446926
step 9151 loss 0.022257840260863304
step 9152 loss 0.02225676365196705
step 9153 loss 0.022255681455135345
step 9154 loss 0.022254614159464836
step 9155 loss 0.022253528237342834
step 9156 loss 0.022252462804317474
step

step 9374 loss 0.022016208618879318
step 9375 loss 0.022015152499079704
step 9376 loss 0.022014088928699493
step 9377 loss 0.022013025358319283
step 9378 loss 0.022011971101164818
step 9379 loss 0.022010894492268562
step 9380 loss 0.022009840235114098
step 9381 loss 0.022008784115314484
step 9382 loss 0.022007720544934273
step 9383 loss 0.022006655111908913
step 9384 loss 0.022005602717399597
step 9385 loss 0.022004537284374237
step 9386 loss 0.022003483027219772
step 9387 loss 0.02200242690742016
step 9388 loss 0.0220013614743948
step 9389 loss 0.022000296041369438
step 9390 loss 0.021999254822731018
step 9391 loss 0.021998194977641106
step 9392 loss 0.02199713885784149
step 9393 loss 0.02199607901275158
step 9394 loss 0.021995024755597115
step 9395 loss 0.02199397049844265
step 9396 loss 0.021992914378643036
step 9397 loss 0.021991867572069168
step 9398 loss 0.021990807726979256
step 9399 loss 0.021989760920405388
step 9400 loss 0.021988699212670326
step 9401 loss 0.02198764123022556

step 9619 loss 0.02176470309495926
step 9620 loss 0.021763717755675316
step 9621 loss 0.021762702614068985
step 9622 loss 0.021761709824204445
step 9623 loss 0.021760690957307816
step 9624 loss 0.021759705618023872
step 9625 loss 0.021758686751127243
step 9626 loss 0.021757710725069046
step 9627 loss 0.021756716072559357
step 9628 loss 0.02175568975508213
step 9629 loss 0.021754708141088486
step 9630 loss 0.021753700450062752
step 9631 loss 0.021752705797553062
step 9632 loss 0.02175169438123703
step 9633 loss 0.02175070531666279
step 9634 loss 0.021749692037701607
step 9635 loss 0.021748702973127365
step 9636 loss 0.021747702732682228
step 9637 loss 0.021746696904301643
step 9638 loss 0.021745696663856506
step 9639 loss 0.021744700148701668
step 9640 loss 0.021743714809417725
step 9641 loss 0.02174270711839199
step 9642 loss 0.021741699427366257
step 9643 loss 0.02174069918692112
step 9644 loss 0.021739698946475983
step 9645 loss 0.021738708019256592
step 9646 loss 0.02173770964145660

step 9872 loss 0.021507473662495613
step 9873 loss 0.021506451070308685
step 9874 loss 0.021505389362573624
step 9875 loss 0.0215043593198061
step 9876 loss 0.021503331139683723
step 9877 loss 0.021502288058400154
step 9878 loss 0.021501250565052032
step 9879 loss 0.02150021493434906
step 9880 loss 0.02149917557835579
step 9881 loss 0.02149813249707222
step 9882 loss 0.021497121080756187
step 9883 loss 0.021496068686246872
step 9884 loss 0.0214950330555439
step 9885 loss 0.021493997424840927
step 9886 loss 0.021492965519428253
step 9887 loss 0.021491922438144684
step 9888 loss 0.02149089053273201
step 9889 loss 0.02148984931409359
step 9890 loss 0.021488819271326065
step 9891 loss 0.021487779915332794
step 9892 loss 0.021486740559339523
step 9893 loss 0.021485701203346252
step 9894 loss 0.02148466557264328
step 9895 loss 0.021483631804585457
step 9896 loss 0.02148258127272129
step 9897 loss 0.021481551229953766
step 9898 loss 0.021480511873960495
step 9899 loss 0.02147948183119297
step